In [1]:
import numpy as np
from itertools import product
from scipy.signal import convolve2d

In [2]:
data = open('../input/D11.txt', 'r', encoding='utf-8').read().splitlines()
seating = np.array([list(r) for r in data])
seating = (seating == '#') * 1 - (seating == '.') * 1 # '.' = floor (-1), 'L' = empty (0), '#' = occupied (1)

In [12]:
def simulate_seating(seating, occupied_tolerance, first_seat=False):
    seat_changes = np.array([1])
    seating = np.copy(seating)
    while seat_changes.any():
        if first_seat:
            adjacent_seats = np.zeros(seating.shape)
            for row, col in product(range(seating.shape[0]), range(seating.shape[1])):
                adjacent_seats[row, col] = np.sum([
                    1 == next((x for x in         np.flip( seating[:row,   col   ])     if x != -1), 0),  # UP
                    1 == next((x for x in                  seating[row+1:, col   ]      if x != -1), 0),  # DOWN
                    1 == next((x for x in         np.flip( seating[row,    :col  ])     if x != -1), 0),  # LEFT
                    1 == next((x for x in                  seating[row,    col+1:]      if x != -1), 0),  # RIGHT
                    1 == next((x for x in np.diag(         seating[row+1:, col+1:])     if x != -1), 0),  # BOTTOM-RIGHT
                    1 == next((x for x in np.diag(np.rot90(seating[row+1:, :col  ], 1)) if x != -1), 0),  # BOTTOM-LEFT
                    1 == next((x for x in np.diag(np.rot90(seating[:row,   :col  ], 2)) if x != -1), 0),  # TOP-LEFT
                    1 == next((x for x in np.diag(np.rot90(seating[:row,   col+1:],-1)) if x != -1), 0),  # TOP-RIGHT  
                ])
        else:
            convolution_matrix = np.array([[1,1,1], [1,0,1], [1,1,1]])
            adjacent_seats = convolve2d(seating > 0, in2=convolution_matrix, boundary='fill', fillvalue=0)[1:-1,1:-1]
        seat_changes = (  (seating == 0) * (adjacent_seats == 0)                  *  1 
                        + (seating == 1)  * (adjacent_seats >= occupied_tolerance) * -1)
        seating += seat_changes
    return (seating > 0).sum()

In [13]:
simulate_seating(seating, occupied_tolerance=4, first_seat=False)

2247

In [14]:
simulate_seating(seating, occupied_tolerance=5, first_seat=True)

2011